In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as m
from scipy.stats import linregress # used to get the linear parameters from the results

In [ ]:
# Parameters
T = 1 # time
alpha = 0.01 # alpha coefficient in the SABR model
rho = 0.25 # correlation between F(t) and sigma(t), rho is an element of [-1, 1]
M=10**4 # Monte Carlo simulations

In [ ]:
# Function will compute the strong error for the implicit EM approximation of the Lamperti transformation
# The Milstein approximation of the SABR model will be used as the reference solution to calculate strong error
def Implicit(T, N, sig_0, M, alpha, Y_0, rho, theta):
    # sig_0 = sigma(0)
    dt = T/N # step size
    Milstein_N = 2**18 # Number of steps for reference solution
    Y_n = np.ones(M)*Y_0 # array to hold values for Y at t for each monte carlo simulation
    sig_t = np.ones(M)*sig_0 # array to hold values for sigma at t for each monte carlo simulation
    F_0 = (Y_0 * (1 - theta))**(1/(1 - theta)) # convert Y(0) to F(0)
    Mil = np.ones(M)*F_0 # array to hold values for Milstein at t for each monte carlo simulation
    A = np.array([alpha*rho, alpha*np.sqrt(1 - rho**2)]) # Cholesky factor
    dt_mil = T/Milstein_N # step size for reference solution
    B = np.zeros(M) # empty array to hold cumulative Brownian motion
    B1_Y = np.zeros(M) # empty array to hold Brownian motion
    for i in range(Milstein_N):
        Z = np.sqrt(dt_mil) * np.random.normal(0, 1, (2, M)) # calculate Brownian increment for B1 and B2
        B += Z[0,:] # cumulate the Brownian increment for B1
        B2 = A[0]*Z[0,:] + A[1]*Z[1,:]  # Brownian motion for B2 in the reference solution
        B1 = Z[0, :] # Brownian motion for B1 in the reference solution
        sig_t = sig_t*np.exp((-alpha**2 / 2)*dt_mil + B2) # sigma(t)
        Mil = Mil + sig_t*B1*np.abs(Mil)**theta +\
            0.5*(sig_t**2)*theta*(B1**2 - dt_mil)*np.abs(Mil)**(2*theta - 1) # Milstein approximation
        
        # to keep on the same path, Yn will be calculated when the below condition is true
        if (i+1)%(Milstein_N//N) == 0:
            Y_n = ((Y_n + sig_t*(B-B1_Y))/2) + \
                np.sqrt((((Y_n + sig_t*(B-B1_Y))**2)/4) - (dt * theta * sig_t**2) / (2 * (1 - theta))) # Yn
            B1_Y = np.copy(B) # Store the cumulative Brownian motion for B1
    
    F_n = (Y_n * (1 - theta))**(1/(1 - theta)) # convet Yn to Fn
    return F_n, Mil # Return the values when n=N

In [ ]:
thetaSim = np.linspace(0.1, 0.9, 9) # theta values to be evaluated
Nsteps = np.array([ 2**6, 2**8, 2**9, 2**10, 2**12]) # Range of step sizes

In [ ]:
Mil_se = pd.DataFrame(columns=['theta=0.1', 'theta=0.2', 'theta=0.3', 'theta=0.4', 'theta=0.5',
                                'theta=0.6', 'theta=0.7', 'theta=0.8', 'theta=0.9', 'N'])
for i in range(len(Nsteps)):
    err = np.zeros(len(thetaSim))
    for j in range(len(thetaSim)):
        F_T, Mil= Implicit(T=T, N=Nsteps[i], M=M, sig_0=0.05, alpha=alpha, Y_0=10, rho=rho, theta=thetaSim[j])
        err[j] = np.mean(np.abs(Mil - F_T)) # Strong error calculation
    Mil_se = Mil_se.append({
        'theta=0.1':err[0],
        'theta=0.2':err[1],
        'theta=0.3':err[2],
        'theta=0.4':err[3],
        'theta=0.5':err[4],
        'theta=0.6':err[5],
        'theta=0.7':err[6],
        'theta=0.8':err[7],
        'theta=0.9':err[8],
        'N':int(Nsteps[i])
    }, ignore_index=True)
Mil_se.index = Mil_se.N
del Mil_se['N']

In [ ]:
# Strong error results
Mil_se

In [ ]:
Dt = T / Nsteps
plt.figure(figsize=(10, 10))
plt.loglog(Dt, error.iloc[:,0],'o', markersize = 10, linestyle = '--', label=r'$\theta$ = 0.1')
plt.loglog(Dt, error.iloc[:,1],'+', markersize = 10, linestyle = '--', label=r'$\theta$ = 0.2')
plt.loglog(Dt, error.iloc[:,2],'1', markersize = 10, linestyle = '--', label=r'$\theta$ = 0.3')
plt.loglog(Dt, error.iloc[:,3],'*', markersize = 10, linestyle = '--', label=r'$\theta$ = 0.4')
plt.loglog(Dt, error.iloc[:,4],'x', markersize = 10, linestyle = '--', label=r'$\theta$ = 0.5')
plt.loglog(Dt, error.iloc[:,5],'d', markersize = 10, linestyle = '--', label=r'$\theta$ = 0.6')
plt.loglog(Dt, error.iloc[:,6],'2', markersize = 10, linestyle = '--', label=r'$\theta$ = 0.7')
plt.loglog(Dt, error.iloc[:,7],'3', markersize = 10, linestyle = '--', label=r'$\theta$ = 0.8')
plt.loglog(Dt, error.iloc[:,8],'4', markersize = 10, linestyle = '--', label=r'$\theta$ = 0.9')
plt.xlabel('h', fontsize=18)
plt.ylabel('Sample average of |F(t) - Fn|', fontsize=18)
plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.legend(fontsize=12)
plt.savefig('Strong_error_convergence_transformation.jpg')

In [ ]:
# calculate the strong order of convergence, gamma, and C
for i in range(len(thetaSim)):
    Mil_gamma, Mil_C = linregress(np.log(Dt), np.log(error.iloc[:,i]))[0:2]
    C_Mil = m.exp(Mil_C)
    print('For theta={}, gamma = {} and C = {}\n'.format(round(thetaSim[i],1), Mil_gamma, C_Mil))